In [ ]:
from MNW import mnw, discrete_mnw
from GDRF import gdrf

from agent import Agent
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from util import random_hospital_groups_demands, social_welfare,max_envy
from functools import partial
import time

%load_ext autoreload
%autoreload 2
plt.rcParams.update({'font.size': 14})
plt.rcParams.update({'figure.autolayout': True})

In [ ]:
def one_iteration_with_welfare_envy(meta_types, N, M):
    agents = [Agent(*random_hospital_groups_demands(meta_types, budget_random=True)) for _ in range(N)]
    supplies = np.random.randint(500*N, 1000*N, size=M)
    
    # Discrete MNW
    start = time.time()
    allocations = discrete_mnw(agents, supplies, meta_types)
    end = time.time()
    DMNW_time = end-start
    
    dmnw_envy = max_envy(agents, allocations, supplies)
    dmnw_sw = social_welfare(agents, allocations, supplies)
    
    
    
    # MNW
    start = time.time()
    allocations = mnw(agents, supplies, meta_types)
    end = time.time()
    MNW_time = end-start
    
    mnw_envy = max_envy(agents, allocations, supplies)
    mnw_sw = social_welfare(agents, allocations, supplies)

    # DRF-MT
    start = time.time()
    allocations = gdrf(agents, supplies, meta_types)
    end = time.time()
    GDRF_time = end-start
    
    gdrf_envy = max_envy(agents, allocations, supplies)
    gdrf_sw = social_welfare(agents, allocations, supplies)
    
    
    return DMNW_time, dmnw_envy, dmnw_sw, MNW_time, mnw_envy, mnw_sw, GDRF_time, gdrf_envy, gdrf_sw

     

In [ ]:
%%time
meta_types= [[0],[1, 2], [3, 4, 5], [6, 7, 8, 9]]

M = sum([len(omega) for omega in meta_types])
Ns = [5, 10, 25, 50, 100, 250, 500, 750, 1000]
# Ns = [5, 10]

iterations=16

DMNW_times_L = []
DMNW_envies_L = []
DMNW_sws_L = []


MNW_times_L = []
MNW_envies_L = []
MNW_sws_L = []



GDRF_times_L = []
GDRF_envies_L = []
GDRF_sws_L = []


##### WITH envy or social welfare #####
for N in Ns:
    print(N)

    results = [one_iteration_with_welfare_envy(meta_types, N, M) for _ in range(iterations)]
    DMNW_times, dmnnw_envies, dmnnw_sws, MNW_times, mnw_envies, mnw_sws, GDRF_times, gdrf_envies, gdrf_sws = zip(*results)
    
    
    DMNW_times_L.append(DMNW_times)
    DMNW_envies_L.append(dmnnw_envies)
    DMNW_sws_L.append(dmnnw_sws)
    
    MNW_times_L.append(MNW_times)
    MNW_envies_L.append(mnw_envies)
    MNW_sws_L.append(mnw_sws)
    
    GDRF_times_L.append(GDRF_times)
    GDRF_envies_L.append(gdrf_envies)
    GDRF_sws_L.append(gdrf_sws)

# Running time comparison

In [ ]:
DMNW_times_L = np.array(DMNW_times_L)
DMNW_mean = DMNW_times_L.mean(axis=1)
DMNW_std = DMNW_times_L.std(axis=1)

MNW_times_L = np.array(MNW_times_L)
MNW_mean = MNW_times_L.mean(axis=1)
MNW_std = MNW_times_L.std(axis=1)

GDRF_times_L = np.array(GDRF_times_L)
GDRF_mean = np.nanmean(GDRF_times_L, axis=1)
GDRF_std = np.nanstd(GDRF_times_L ,axis=1)

In [ ]:
plt.plot(Ns,DMNW_mean)
plt.fill_between(Ns, DMNW_mean+DMNW_std, DMNW_mean-DMNW_std, alpha=0.5)
plt.plot(Ns,MNW_mean)
plt.fill_between(Ns, MNW_mean+MNW_std, MNW_mean-MNW_std, alpha=0.5)
plt.plot(Ns,GDRF_mean)
plt.fill_between(Ns, GDRF_mean+GDRF_std, GDRF_mean-GDRF_std, alpha=0.5)


plt.legend(["Discrete MNW", "MNW", "DRF-MT"])
plt.ylabel("seconds")
plt.xlabel("Number of Agents")

In [ ]:
envy_df = pd.DataFrame({"Discrete MNW": np.array(DMNW_envies_L).flatten(),
                        "MNW":          np.array( MNW_envies_L).flatten(),
                        "DRF-MT":         np.array(GDRF_envies_L).flatten(),
                        "Number of agents": np.array([[N]*iterations for N in Ns]).flatten()})

envy_df = pd.melt(envy_df, id_vars=['Number of agents'], value_vars=['Discrete MNW', 'MNW', "DRF-MT"],
        var_name='Algorithm', value_name='Max Envy')

In [ ]:
ax = sns.boxplot(x="Number of agents", y="Max Envy", hue="Algorithm",              
data=envy_df[envy_df["Number of agents"].isin([50,100, 250, 500, 750, 1000])])
# ax = sns.boxplot(x="Number of agents", y="Max Envy", hue="Algorithm",              
# data=envy_df[envy_df["Number of agents"].isin(Ns)])

# Social welfare comparison
## DRF-MT vs Discrete MNW

In [ ]:
dmnw_sw_df = pd.DataFrame(DMNW_sws_L)
gdrf_sw_df = pd.DataFrame(GDRF_sws_L)
normalized_diff = (gdrf_sw_df-dmnw_sw_df)/dmnw_sw_df
normalized_diff.index = Ns

In [ ]:
hist_plot = pd.DataFrame(normalized_diff.values.flatten()).plot(kind="hist", bins=25, title="", legend=False, grid=True)
plt.ylabel("Count")
plt.xlabel("Ratio")

In [ ]:
normalized_diff.T.plot.box()
plt.xlabel("Number of Agents")
plt.ylabel("Ratio")
plt.title("Normalized Difference in Social Welfare")

## Discrete MNW vs MNW

In [ ]:
mnw_sw_df = pd.DataFrame(MNW_sws_L)
normalized_diff_dmnw_mnw = (mnw_sw_df-dmnw_sw_df)/dmnw_sw_df
pd.DataFrame(normalized_diff_dmnw_mnw.values.flatten()).plot(title="",grid=True,legend=False, kind="hist", bins=25)
plt.ylabel("Count")
plt.xlabel("Ratio")

# More running time comparisons

In [ ]:
def one_iteration(meta_types, N, M):
    agents = [Agent(*random_hospital_groups_demands(meta_types, budget_random=True)) for _ in range(N)]
    supplies = np.random.randint(500*N, 1000*N, size=M)
    
    # Discrete MNW
    start = time.time()
    allocations = discrete_mnw(agents, supplies, meta_types)
    end = time.time()
    DMNW_time = end-start
    # MNW

    start = time.time()
    allocations = mnw(agents, supplies, meta_types)
    end = time.time()
    MNW_time = end-start

    # DRF
    start = time.time()
    allocations = gdrf(agents, supplies, meta_types)
    end = time.time()
    GDRF_time = end-start
    
    
    return DMNW_time, MNW_time, GDRF_time

## Bigger meta-types structure

In [ ]:
%%time
meta_types= [[0],[1, 2], [3, 4, 5], [6, 7, 8, 9], [10,11,12,13,14]]

M = sum([len(omega) for omega in meta_types])
Ns = [5, 10, 25, 50, 100, 250, 500, 750, 1000]
# Ns = [5, 10]

iterations=16

bDMNW_times_L = []
bMNW_times_L = []
bGDRF_times_L = []


for N in Ns:
    print(N)
    bDMNW_times = []
    bMNW_times = []
    bGDRF_times = []


#     with Pool(processes=4) as pool:
#         results = pool.starmap(one_iteration, zip([meta_types]*iterations, [N]*iterations, [M]*iterations))
    results = [one_iteration(meta_types, N, M) for _ in range(iterations)]
    bDMNW_times, bMNW_times, bGDRF_times = zip(*results)
    
    bDMNW_times_L.append(bDMNW_times)
    bMNW_times_L.append(bMNW_times)
    bGDRF_times_L.append(bGDRF_times)

In [ ]:
bDMNW_times_L = np.array(bDMNW_times_L)
bDMNW_mean = bDMNW_times_L.mean(axis=1)
bDMNW_std = bDMNW_times_L.std(axis=1)


bMNW_times_L = np.array(bMNW_times_L)
bMNW_mean = bMNW_times_L.mean(axis=1)
bMNW_std = bMNW_times_L.std(axis=1)

bGDRF_times_L = np.array(bGDRF_times_L)
bGDRF_mean = bGDRF_times_L.mean(axis=1)
bGDRF_std = bGDRF_times_L.std(axis=1)

In [ ]:
plt.plot(Ns,bDMNW_mean)
plt.fill_between(Ns, bDMNW_mean+bDMNW_std, bDMNW_mean-bDMNW_std, alpha=0.5)
plt.plot(Ns,bMNW_mean)
plt.fill_between(Ns, bMNW_mean+bMNW_std, bMNW_mean-bMNW_std, alpha=0.5)
plt.plot(Ns,bGDRF_mean)
plt.fill_between(Ns, bGDRF_mean+bGDRF_std, bGDRF_mean-bGDRF_std, alpha=0.5)


plt.legend(["Discrete MNW", "MNW", "DRF-MT"])
plt.ylabel("seconds")
plt.xlabel("Number of Agents")

## Increasing number of resource meta-types

In [ ]:
%%time
Ls = np.array([5, 30, 60, 90])
# Ls = np.array([5, 10])
iterations=16

mt_MNW_times_L = []
mt_GDRF_times_L = []
mt_DMNW_times_L = []

N = 50

for L in Ls:
    print(L)
    meta_types = [list(range(l*5, l*5+5)) for l in range(L)]
    M = L*5

    results = [one_iteration(meta_types, N, M) for _ in range(iterations)]
    DMNW_times, MNW_times, GDRF_times = zip(*results)
    
    mt_DMNW_times_L.append(DMNW_times)
    mt_MNW_times_L.append(MNW_times)
    mt_GDRF_times_L.append(GDRF_times)


In [ ]:
mt_DMNW_times_L = np.array(mt_DMNW_times_L)
mt_DMNW_mean = mt_DMNW_times_L.mean(axis=1)
mt_DMNW_std = mt_DMNW_times_L.std(axis=1)


mt_MNW_times_L = np.array(mt_MNW_times_L)
mt_MNW_mean = mt_MNW_times_L.mean(axis=1)
mt_MNW_std = mt_MNW_times_L.std(axis=1)

mt_GDRF_times_L = np.array(mt_GDRF_times_L)
mt_GDRF_mean = mt_GDRF_times_L.mean(axis=1)
mt_GDRF_std = mt_GDRF_times_L.std(axis=1)

In [ ]:
plt.plot(Ls,mt_DMNW_mean)
plt.fill_between(Ls, mt_DMNW_mean+mt_DMNW_std, mt_DMNW_mean-mt_DMNW_std, alpha=0.5)
plt.plot(Ls,mt_MNW_mean)
plt.fill_between(Ls, mt_MNW_mean+mt_MNW_std, mt_MNW_mean-mt_MNW_std, alpha=0.5)
plt.plot(Ls,mt_GDRF_mean)
plt.fill_between(Ls, mt_GDRF_mean+mt_GDRF_std, mt_GDRF_mean-mt_GDRF_std, alpha=0.5)


plt.legend(["Discrete MNW", "MNW", "DRF-MT"])
plt.ylabel("seconds")
plt.xlabel("Number of meta types")

## Just DRF-MT and MNW

### Scaling up number of agents

In [ ]:
def one_iteration_2methods(meta_types, N, M):
    agents = [Agent(*random_hospital_groups_demands(meta_types, budget_random=True)) for _ in range(N)]
    supplies = np.random.randint(500*N, 1000*N, size=M)

    # MNW
    start = time.time()
    allocations = mnw(agents, supplies, meta_types)
    end = time.time()
    MNW_time = end-start

    # DRF
    start = time.time()
    allocations = gdrf(agents, supplies, meta_types)
    end = time.time()
    GDRF_time = end-start
    
    
    return MNW_time, GDRF_time

In [ ]:
%%time
meta_types= [[0],[1, 2], [3, 4, 5], [6, 7, 8, 9]]

M = sum([len(omega) for omega in meta_types])
Ns = [50, 500, 1000, 2000, 4000, 7000, 10000]
# Ns = [50, 500]

iterations=16


MNW_2_times_L = []
MNW_2_envies_L = []
MNW_2_sws_L = []



DRFMT_2_times_L = []
DRFMT_2_envies_L = []
DRFMT_2_sws_L = []



#### WITHOUT envy or social welfare #####
for N in Ns:
    print(N)

#     with Pool(processes=4) as pool:
#         results = pool.starmap(one_iteration, zip([meta_types]*iterations, [N]*iterations, [M]*iterations))
    results = [one_iteration_2methods(meta_types, N, M) for _ in range(iterations)]
    MNW_times, DRFMT_times = zip(*results)
    

    MNW_2_times_L.append(MNW_times)
    DRFMT_2_times_L.append(DRFMT_times)
    



In [ ]:
MNW_2_times_L = np.array(MNW_2_times_L)
MNW_2_mean = MNW_2_times_L.mean(axis=1)
MNW_2_std = MNW_2_times_L.std(axis=1)

DRFMT_2_times_L = np.array(DRFMT_2_times_L)
DRFMT_2_mean = DRFMT_2_times_L.mean(axis=1)
DRFMT_2_std = DRFMT_2_times_L.std(axis=1)

In [ ]:
plt.plot(Ns,MNW_2_mean)
plt.fill_between(Ns, MNW_2_mean+MNW_2_std, MNW_2_mean-MNW_2_std, alpha=0.5)
plt.plot(Ns,DRFMT_2_mean)
plt.fill_between(Ns, DRFMT_2_mean+DRFMT_2_std, DRFMT_2_mean-DRFMT_2_std, alpha=0.5)


plt.legend(["MNW", "DRF-MT"])
plt.ylabel("seconds")
plt.xlabel("Number of Agents")

### Scaling up meta-types

In [ ]:
%%time
Ls = np.array([50, 250, 500, 1000, 1500])
# Ls = np.array([50, 100])
iterations=16

mt_MNW_2_times_L = []
mt_DRFMT_2_times_L = []


N = 50

for L in Ls:
    print(L)
    meta_types = [list(range(l*5, l*5+5)) for l in range(L)]
    M = L*5

    results = [one_iteration_2methods(meta_types, N, M) for _ in range(iterations)]
    MNW_2_times, DRFMT_2_times = zip(*results)
    
    mt_MNW_2_times_L.append(MNW_2_times)
    mt_DRFMT_2_times_L.append(DRFMT_2_times)




In [ ]:
mt_MNW_2_times_L = np.array(mt_MNW_2_times_L)
mt_MNW_2_mean = mt_MNW_2_times_L.mean(axis=1)
mt_MNW_2_std = mt_MNW_2_times_L.std(axis=1)

mt_DRFMT_2_times_L = np.array(mt_DRFMT_2_times_L)
mt_DRFMT_2_mean = mt_DRFMT_2_times_L.mean(axis=1)
mt_DRFMT_2_std = mt_DRFMT_2_times_L.std(axis=1)

In [ ]:
plt.plot(Ls,mt_MNW_2_mean)
plt.fill_between(Ls, mt_MNW_2_mean+mt_MNW_2_std, mt_MNW_2_mean-mt_MNW_2_std, alpha=0.5)
plt.plot(Ls,mt_DRFMT_2_mean)
plt.fill_between(Ls, mt_DRFMT_2_mean+mt_DRFMT_2_std, mt_DRFMT_2_mean-mt_DRFMT_2_std, alpha=0.5)


plt.legend(["MNW", "DRF-MT"])
plt.ylabel("seconds")
plt.xlabel("Number of meta types")